<a href="https://colab.research.google.com/github/alessandra-barbosa/blocker_fraud_company/blob/main/AB_blocker_fraud_company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Cardio Image](https://plus.unsplash.com/premium_photo-1663127034054-4d4e1d8712e2?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxzZWFyY2h8N3x8ZnJhdWR8ZW58MHx8MHx8&auto=format&fit=crop&w=500&q=60)
<h1>Blocker fraud company </h1>

## 0.0 - Business undertanding

**The Blocker Fraud Company**

- Financial transactions' fraud detection specialized company.
- The Blocker Fraud service ensures fraudulent transactions block.
- Business Model: service's performance monetization.

**Expansion Strategy in Brazil**

1. The company receives 25% of each transaction value truly detected as fraud.
2. The company receives 5% of each transaction value detected as fraud, however the transaction is legitimate.
3. The company gives back 100% of the value for the customer in each transaction detected as legitimate, however the transaction is actually a fraud.

**Goal**

- Create a model with high accuracy and precision with respect to transactions' fraud detection.

**Deliverables**

- Deployed model with API access. The API must inform "Fraud" or "Legitimate" when the transaction is inputed.
- A Readme about how to use the tool.
- Model performance and results report with respect to profit and loss. The following questions must be answered:

    - What is the model's precision and accuracy?
    - What is the model's reliability with respect to transactions' classification as legitimate or fraudulent?
    - What is the company's forecasted revenue if the model classifies 100% of the transactions?
    - What is the company's forecasted loss in case of model's failure?
    - What is the Blocker Fraud Company forecasted profit using the model?

## 1.0 - Imports

In [21]:
#!pip install inflection

### 1.1 - Libraries

In [3]:
import pandas as pd
import numpy as np
import inflection
from scipy.stats import kurtosis, skew
from random import sample as sp
from pandas_profiling      import ProfileReport #conda install -c conda-forge ydata-profiling

# ## Database 
from sqlalchemy import create_engine
import sqlite3

# ## Data Transformation
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OrdinalEncoder,OneHotEncoder

# ## Visualization
# import seaborn as sns
# from seaborn import pairplot
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
# from scipy.stats import pointbiserialr

# ## Machine Learning, selection and split
from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,ExtraTreesClassifier
# from sklearn.feature_selection import RFE
# from sklearn.decomposition          import PCA
# from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score
# from sklearn.linear_model import LogisticRegression 
# from xgboost import XGBClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score,RandomizedSearchCV,GridSearchCV
# import pickle

# ## Configurations
# from IPython.display import Image
# from IPython.display import HTML
# import warnings



<ipython-input-3-e58f32863bd9>:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling      import ProfileReport #conda install -c conda-forge ydata-profiling


### 1.2 - Funcions

In [23]:
# # Calculating Scores to classification 
# def metrics_score(y_valid, y_hat, mdl_name):
#     '''
#         y_hat:    is the prediction array
#         y_valid:  is the true targe array or series
#         mdl_name: is the name of machine learning model
        
#         Accuracy formula  -->   accuracy   = np.mean(y_hat == y_valid)
#         Precision formula -->   precision  = positive_true/(positive_true+positive_false)      
#         Recall formula    -->   recall     = positive_true/(positive_true+negative_false)
        
#         Return -->> mdl_name, accuracy, precision, recall
        
#     '''
#     accuracy = accuracy_score(y_valid, y_hat)
#     precision = precision_score(y_valid, y_hat)
#     recall = recall_score(y_valid, y_hat)
#     F1_score = f1_score(y_valid, y_hat)

    
#     return pd.DataFrame([[mdl_name, accuracy, precision, recall,F1_score]], 
#                         columns=['mdl_name', 'accuracy', 'precision', 'recall', 'F1_score'
#                                  ])
    


# # Jupyter configuring to better experience
# def jupyter_settings():
#     %matplotlib inline
#     %pylab inline
        
#     plt.style.use('bmh')
#     plt.rcParams['figure.figsize'] = [25,9]
#     plt.rcParams['font.size'] = 24
    
#     display(HTML('<style>.container{width:100% !important; }</style>'))
#     pd.options.display.max_columns = None
#     pd.options.display.max_rows = None
#     pd.set_option('display.expand_frame_repr', False)
    
#     sns.set()
# jupyter_settings()  


# def highlight_max(s):
#     '''
#     highlight the maximum in a Series yellow.
#     '''
#     is_max = s == s.max()
#     return ['background-color: yellow' if v else '' for v in is_max]

# # To hide warning messages
# warnings.filterwarnings('ignore')


### 1.3 - Data Collect - CSV

In [4]:
## Loading dataset
# from google import colab as cl
# file_upload=cl.files.upload()
df_raw = pd.read_csv('/content/PS_20174392719_1491204439457_log.csv', sep=',')
#df_raw = pd.read_csv(r'C:\Users\Utente77\repos\blocker_fraud_company_PA002\datasets\PS_20174392719_1491204439457_log.csv', sep=',')

In [5]:
df_raw.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### 1.5 - Creating Datawerehouse

##### 1.5.1 - Database

In [7]:
# # creating database
# path_local="C:/Users/Utente77/repos/blocker_fraud_company_PA002/datasets/"
# engine_db = create_engine(f"sqlite:////{path_local}/ccd.db", echo=False)
# conn = sqlite3.Connection('blocker_fraud.db')
# cursor = conn.cursor()

# # create table
# query = '''CREATE TABLE IF NOT EXISTS fraud(
#                                 step INTEGER,
#                                 type TEXT,
#                                 amount INTEGER,
#                                 nameOrig TEXT,
#                                 oldbalanceOrg INTEGER,
#                                 newbalanceOrig INTEGER,
#                                 nameDest TEXT,
#                                 oldbalanceDest INTEGER,
#                                 newbalanceDest INTEGER,
#                                 isFraud INTEGER,
#                                 isFlaggedFraud INTEGER
                                
#                                 )'''

# cursor.execute(query)
# #engine = engine_db()
# df_raw.to_sql('fraud', con=conn, if_exists='append', index=False)



6362620

#### 1.5.2 - Data Collect from Database

In [ ]:
# # Selecting data
# query="""
#     SELECT *
#     FROM fraud;

# """
# df=pd.read_sql_query(query,conn)


##### 1.5.3 - Rename Columns

In [6]:
# Old columns
cols_old=df_raw.columns
cols_old

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [7]:
# apply snakecase to columns names
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
# rename
df_raw.columns = cols_new
df_raw.columns

Index(['step', 'type', 'amount', 'name_orig', 'oldbalance_org',
       'newbalance_orig', 'name_dest', 'oldbalance_dest', 'newbalance_dest',
       'is_fraud', 'is_flagged_fraud'],
      dtype='object')

### 1.6  ***Cheking balance data***
- Proportion , balanced data

In [8]:
# check type feature proportion
df_raw['type'].value_counts(normalize=True)


CASH_OUT    0.351663
PAYMENT     0.338146
CASH_IN     0.219923
TRANSFER    0.083756
DEBIT       0.006512
Name: type, dtype: float64

In [10]:
# check type feature proportion
df_raw['is_fraud'].value_counts(normalize=True)


0    0.998709
1    0.001291
Name: is_fraud, dtype: float64

In [11]:
# check type feature proportion
df_raw['is_flagged_fraud'].value_counts(normalize=True)


0    0.999997
1    0.000003
Name: is_flagged_fraud, dtype: float64

In [12]:
# check type feature proportion
df_raw['type'].value_counts(normalize=True)

CASH_OUT    0.351663
PAYMENT     0.338146
CASH_IN     0.219923
TRANSFER    0.083756
DEBIT       0.006512
Name: type, dtype: float64

### 1.7 Apply stratified sampling

In [13]:
# apply stratified sampling

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_1_index, train_2_index in split.split(df_raw, df_raw['is_fraud']):
    strat_train_set = df_raw.loc[train_1_index]
    strat_test_set = df_raw.loc[train_2_index]


In [16]:
strat_train_set.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4453834 entries, 4310249 to 1541412
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   step              int64  
 1   type              object 
 2   amount            float64
 3   name_orig         object 
 4   oldbalance_org    float64
 5   newbalance_orig   float64
 6   name_dest         object 
 7   oldbalance_dest   float64
 8   newbalance_dest   float64
 9   is_fraud          int64  
 10  is_flagged_fraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 407.8+ MB


In [17]:
strat_test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908786 entries, 1453282 to 5365974
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   step              int64  
 1   type              object 
 2   amount            float64
 3   name_orig         object 
 4   oldbalance_org    float64
 5   newbalance_orig   float64
 6   name_dest         object 
 7   oldbalance_dest   float64
 8   newbalance_dest   float64
 9   is_fraud          int64  
 10  is_flagged_fraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 174.8+ MB


## 2.0 - Descriptive Analysis

In [22]:
# load sampled set for future notebook work
df = pd.read_csv('/content/train_set_v02.csv', sep=',')
#df = pd.read_csv('datasets/train_set_v02.csv')
df.tail()

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud
160453,253,CASH_OUT,204701.03,C1622157331,0.00,0.00,C220146073,507816.01,712517.04,0.0,0.0
160454,253,PAYMENT,9505.98,C923963130,7312.00,0.00,M161644108,0.00,0.00,0.0,0.0
160455,298,CASH_IN,132753.95,C540599726,24002210.81,24134964.76,C307127360,1186205.98,1053452.03,0.0,0.0
160456,232,CASH_OUT,315435.00,C462907113,125.00,0.00,C1254839494,0.00,315435.00,0.0,0.0
160457,236,CASH_IN,220219.03,C1082782647,6085277.00,NaN,NaN,NaN,NaN,NaN,NaN


### 2.1 - Dimensions

In [23]:
# Printing the data dimensions, how much rows and columns
print(f'{df.shape[0]} linhas.')
print(f'{df.shape[1]} colunas.')

160458 linhas.
11 colunas.


### 2.2 - Datatype and NaN fillout

In [24]:
# Columns Datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160458 entries, 0 to 160457
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   step              160458 non-null  int64  
 1   type              160458 non-null  object 
 2   amount            160458 non-null  float64
 3   name_orig         160458 non-null  object 
 4   oldbalance_org    160458 non-null  float64
 5   newbalance_orig   160457 non-null  float64
 6   name_dest         160457 non-null  object 
 7   oldbalance_dest   160457 non-null  float64
 8   newbalance_dest   160457 non-null  float64
 9   is_fraud          160457 non-null  float64
 10  is_flagged_fraud  160457 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 13.5+ MB


In [26]:
aux=df.isna().sum().sort_values(ascending=False)
aux1=df.isna().sum().sort_values(ascending=False)/df.shape[0]*100

na=pd.concat([aux,aux1],axis=1)
na.columns=['NaN', 'NaN %']
na

,NaN,NaN %
newbalance_orig,1,0.000623
name_dest,1,0.000623
oldbalance_dest,1,0.000623
newbalance_dest,1,0.000623
is_fraud,1,0.000623
is_flagged_fraud,1,0.000623
step,0,0.000000
type,0,0.000000
amount,0,0.000000
name_orig,0,0.000000


In [ ]:
#Image('/content/Cardio_Disease.png')
Image(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\img\Cardio_Disease.png')

### 2.3 - Data Description

In [ ]:
# profile=ProfileReport(df, title="Pandas Profiling Report")
# profile.to_file(r"C:\Users\Utente77\repos\cardio_catch_disease_PA001\img\report.html")
# profile

In [ ]:
# Descriptive statistics 

describe = df.describe().T.drop('id', axis=0)
describe['range'] = describe['max'] - describe['min']
describe['median'] = df.apply( np.median )
describe['kurtosis'] = df.apply( kurtosis )
describe['skew'] = df.apply( skew )
describe

### ***Problematics***
- age in days
- hight_pressure min above 0
- low_pressure min above 



In [ ]:
# select categorical attributes
cat_attr = df.select_dtypes(exclude=['int64', 'float64'])

In [ ]:
cat_attr.value_counts()

In [ ]:
a=pd.DataFrame(df['is_fraud'].value_counts(normalize=False))
b=pd.DataFrame(df['is_fraud'].value_counts(normalize=True))
c=pd.DataFrame(df['is_flagged_fraud'].value_counts(normalize=False))
d=pd.DataFrame(df['is_flagged_fraud'].value_counts(normalize=True))
pd.concat([a,c,b,d],axis=1)

## 3.0 - Feature Engineering

In [ ]:
df1=df.copy()

In [ ]:
# age - change days to year old
df1['age'] = (df1['age'] / 365).astype('int')

# height - change centimeters to meters
df1['height'] = (df1['height'] / 100)

<h1>BMI Levels</h1>

![Cardio Image](https://www.builtlean.com/wp-content/uploads/2013/06/Bmi-chart-1.jpg)


In [ ]:
# BMI calculus and slices in four levels
df1['bmi'] = (df1['weight']/(df1['height'] * df1['height']))
df1['bmi_level'] = df1['bmi'].apply(lambda x: 
                                              0 if x <= 18.5 else
                                              1      if x >  18.5 and x <= 24.9 else
                                              2  if x >= 25.0 and x <= 29.9 else
                                              3)

<h1>Diastolic and systolic blood pressure</h1>

![Cardio Image](https://www.bloomsburysurgery.nhs.uk/media/content/images/bp-range-chart.jpg)


In [ ]:
# Hypertension and Hipotension binary
df1['hypertension'] = df1.apply(lambda x: 1 if x[4] >=140 and x[5] >= 90 else 0, axis=1)
df1['hypotension'] = df1.apply(lambda x: 1 if x[4] <=90 and x[5] <= 60 else 0, axis=1)

In [ ]:
# A difference between systolic and diastolic arterial pressure, 

df1['difference'] = df1['hight_pressure'] - df1['low_pressure'] # A difference between systolic and diastolic arterial pressure
df1['convergent'] = df1['difference'].apply(lambda x: 1 if x <= 30 else 0) # lesser than 30mmhg called convergent 
df1['divergent'] = df1['difference'].apply(lambda x: 1 if x >= 60 else 0)# and upper than 60mmhg called divergent 

In [ ]:
# Slicing ages with three ranges
df1['age_range'] = df1['age'].apply(lambda x: 1    if x <= 50 
                                                   else 2   if x >= 51 and x <= 59 
                                                   else 3 if x >= 60 
                                                   else x)

In [ ]:
df1['weight_range'] = df1['weight'].apply(
    lambda x:   0.50 if x <= 50  else
                0.75 if x >= 51  and x <= 75 else
                1.00 if x >= 76  and x <= 100 else
                1.25 if x >= 101 and x <= 125 else
                1.50 if x >= 126 and x <= 150 else
                1.75 if x >= 151 and x <= 175 else 2.00)

In [ ]:
# Descriptive statistics after feature engineer 

describe = df1.describe().T.drop('id', axis=0)
describe['range'] = describe['max'] - describe['min']
describe['median'] = df1.apply( np.median )
describe['kurtosis'] = df1.apply( kurtosis )
describe['skew'] = df1.apply( skew )
describe

## 4.0 - Data Filtering

In [ ]:
# sets plot size
plt.figure(figsize=(16,6))
# plots scatterplot
plt.subplot(1,3,1)
sns.scatterplot(x='height', y='weight', data=df1, hue='age_range', palette='rainbow', alpha=0.75)
plt.xlabel("Height (m)")
plt.ylabel("Weight (kg)")
plt.title("Weight x Height", fontsize=14)

plt.subplot(1,3,2)
plt.title('Height')
sns.boxplot(df1['height']);

plt.subplot(1,3,3)
plt.title('Weight')
sns.boxplot(df1['weight']);

# displays the plot
plt.show()

In [ ]:
# sets plot size
plt.figure(figsize=(14,8))

# plots scatterplot
plt.subplot(1,2,1)
sns.scatterplot(x='age', y='height', data=df1, hue='weight', size='weight', alpha=0.75)
plt.xlabel("Age (years)")
plt.ylabel("Height (m)")
plt.title("Height x Age", fontsize=14)

# plots scatterplot
plt.subplot(1,2,2)
sns.scatterplot(x='age', y='weight', data=df1, hue='height', size='height', alpha=0.75)
plt.xlabel("Age (years)")
plt.ylabel("Weight (kg)")
plt.title("Weight x Age", fontsize=14)

# displays the plot
plt.show()

In [ ]:
# sets plot size
plt.figure(figsize=(16,12))

# plots scatterplot
plt.subplot(2,2,1)
sns.scatterplot(x='age', y='hight_pressure', data=df1, hue='weight', size='weight', alpha=0.75)
plt.xlabel("Age (years)")
plt.ylabel("Systolic blood pressure")
plt.title("Systolic x Age", fontsize=14)

plt.subplot(2,2,3)
sns.scatterplot(x='age', y='low_pressure', data=df1, hue='weight', size='weight', alpha=0.75)
plt.xlabel("Age (years)")
plt.ylabel("Diastolic blood pressure")
plt.title("Diastolic x Age", fontsize=14)

plt.subplot(2,2,2)
plt.title('Hight Pressure')
sns.boxplot(df1['hight_pressure']);

plt.subplot(2,2,4)
plt.title('Low Pressure')
sns.boxplot(df1['low_pressure']);

# displays the plot
plt.show()

In [ ]:
# Boxplot from numerical data
plt.figure(figsize=(16,6))

plt.subplot(1,2,1)
plt.title('Difference')
sns.boxplot(df1['difference']);

plt.subplot(1,2,2)
plt.title('BMI')
sns.boxplot(df1['bmi']);



#### 2.3.1  Inconsistent numbers of **hight pressure** and **low pressure** 


In [ ]:
# calculates Q1, Q3 and IQR
Q1 = np.quantile(df1['hight_pressure'], .25)
Q3 = np.quantile(df1['hight_pressure'], .75)
IQR = Q3 - Q1

# calculates the outliers boundaries through statistical relationship
low_outliers_h = Q1 - 1.5 * IQR
high_outliers_h = Q3 + 1.5 * IQR

print("Systolic boundaries for outliers")
print(f"Q1 = {Q1}\nQ3 = {Q3}\nIQR = {IQR}\nlow_outliers = {low_outliers_h}\nhigh_outliers = {high_outliers_h}")


# calculates Q1, Q3 and IQR
Q1 = np.quantile(df1['low_pressure'], .25)
Q3 = np.quantile(df1['low_pressure'], .75)
IQR = Q3 - Q1

# calculates the outliers boundaries through statistical relationship
low_outliers_l = Q1 - 1.5 * IQR
high_outliers_l = Q3 + 1.5 * IQR

print("\nDiastolic boundaries for outliers")
print(f"Q1 = {Q1}\nQ3 = {Q3}\nIQR = {IQR}\nlow_outliers = {low_outliers_l}\nhigh_outliers = {high_outliers_l}")

In [ ]:

print(f'''{df1[(df1['low_pressure'] < low_outliers_l) | (
    df1['low_pressure'] > high_outliers_l)].shape[0]} inconsistent rows on low_pressure.''')
print(f'''{df1[(df1['hight_pressure'] < low_outliers_h) | (
    df1['hight_pressure'] > high_outliers_h)].shape[0]} inconsistent rows on hight_pressure. ''')

# #filtering data without inconsistence data
df1 = df1[(df1['low_pressure'] >= 20) & (df1['low_pressure'] <= 150)]
df1 = df1[(df1['hight_pressure'] >= 40) & (df1['hight_pressure'] <= 300)]

## 5.0 - EDA - Explolatory Data Analysis

In [ ]:
df2=df1.copy()


### 5.1 - Univariate Analysis

In [ ]:
# Distribution from all data
df2.hist(bins=50);

In [ ]:
# Distribution from all data
plt.subplot(2,4,1)
plt.title('age')
sns.boxplot(df2['age']);

plt.subplot(2,4,2)
plt.title('height')
sns.boxplot(df2['height']);

plt.subplot(2,4,3)
plt.title('weight')
sns.boxplot(df2['weight']);

plt.subplot(2,4,4)
plt.title('low_pressure')
sns.boxplot(df2['low_pressure']);

plt.subplot(2,4,5)
plt.title('hight_pressure')
sns.boxplot(df2['hight_pressure']);

plt.subplot(2,4,6)
plt.title('bmi')
sns.boxplot(df2['bmi']);

plt.subplot(2,4,7)
plt.title('difference')
sns.boxplot(df2['difference']);


In [ ]:
df2.columns

In [ ]:
df2.isna().sum()

### 5.2 - Bivariate Analysis

##### 5.2.1 - How more active, less diseases
False

In [ ]:
df2[['cardio','active']].groupby('cardio').value_counts(normalize=True ).reset_index()

In [ ]:
subplot(1,2,1)
sns.countplot(hue='cardio', x='active', data=df2);

subplot(1,2,2)
sns.heatmap(df2[['cardio','active']].corr(method='pearson'), annot=True);


##### 5.2.2 - Cholesterou over level I, more risk
 True (perarson >0.2)

In [ ]:
df2[['cardio','cholesterol']].groupby('cardio').value_counts(normalize=True ).reset_index()

In [ ]:
plt.subplot(1,2,1)
sns.countplot(hue='cardio', x='cholesterol', data=df2);

plt.subplot(1,2,2)
sns.heatmap(df2[['cardio','cholesterol']].corr(method='pearson'), annot=True);


##### 5.2.3 - How more clucose, more risk
   False

The correlation it's so low, insignificat to model

In [ ]:
plt.subplot(1,2,1)
sns.countplot(hue='cardio', x='glucose', data=df2);

plt.subplot(1,2,2)
sns.heatmap(df2[['cardio','glucose']].corr(method='pearson'), annot=True);

##### 5.2.4 - How more older, more risk
   True (perarson >0.2)

following the years it's more count and more avarage pear year old, low correlation, but, probly it's important to model

In [ ]:
subplot(2,2,1)
plt.title('Counting')
sns.countplot(x='age', hue='cardio', data=df2);

subplot(2,2,2)
plt.title('Average')
sns.lineplot(data=df2, x='age', y='cardio', estimator='mean');

subplot(2,2,3)
plt.title('Distribution')
sns.distplot(df2['age']);

subplot(2,2,4)
plt.title('Correlation Between Cardio vs Age')
sns.heatmap(df2[['cardio', 'age']].corr('pearson'), annot=True);

##### 5.2.5 - Over 60 year old with Hypertension, more risk
   False

In [ ]:
aux = df2[['age_range', 'cardio']].groupby('age_range').mean().reset_index()
aux1 = df2[['cardio', 'age_range', 'hypertension']]

plt.subplot(2,2,1)
sns.countplot(hue='cardio', x='hypertension', data=aux1);

plt.subplot(2,2,2)
sns.countplot(x='age', hue='cardio', data=df2);

# Average incidence of diseases by hypertension and over 60 year.
subplot(2,2,3)
aux = df2[['hypertension', 'cardio', 'age_range']].groupby(['hypertension', 'age_range']).mean().reset_index()
sns.barplot(x='age_range', y='cardio', hue='hypertension', data=aux);

subplot(2,2,4)
sns.heatmap(df2[['cardio', 'hypertension', 'age']].corr('pearson'), annot=True);

##### 5.2.6 - 60 year old with Hypotension, more risk
   False

In [ ]:
aux = df2[['hypotension', 'age_range', 'cardio']].groupby(
    ['hypotension', 'age_range']).mean().reset_index()

plt.subplot(1,2,1)
sns.barplot(x='age_range', y='cardio', hue='hypotension', data=aux);


plt.subplot(1,2,2)
sns.heatmap(df2[['age_range', 'hypotension', 'cardio']].corr('pearson'), annot=True);

##### 5.2.7 - 60 year old, more risk
   False

In [ ]:
df2[['cardio','age_range']].groupby('cardio').value_counts(normalize=True ).reset_index()

##### 5.2.8 - Divergent AP, more risk
   True - Relevant variable

In [ ]:
aux = df2[['divergent', 'cardio']].groupby('divergent').mean().reset_index()

plt.subplot(1,2,1)
sns.barplot(x='divergent', y='cardio', data=aux)

plt.subplot(1,2,2)
sns.heatmap(df2[['cardio', 'divergent']].corr('pearson'), annot=True)

##### 5.2.9 - Convergent AP, more risk
   False

In [ ]:
plt.subplot(1,2,1)
sns.countplot(x='convergent', hue='cardio',
    data=df2[['cardio', 'convergent']].groupby('convergent').mean().reset_index());

plt.subplot(1,2,2)
sns.heatmap(df2[['cardio', 'convergent']].corr('pearson'), annot=True);

##### 5.2.10 - Hypertension, more risk
   True

Average incidence by Hypertension

In [ ]:
aux = df2[['hypertension', 'cardio']].groupby('hypertension').mean().reset_index()

plt.subplot(1,3,1)
sns.barplot(x='hypertension', y='cardio', data=aux);

plt.subplot(1,3,2)
sns.countplot(x='hypertension', hue='cardio', data=df2);

plt.subplot(1,3,3)
sns.heatmap(df2[['hypertension', 'cardio']].corr('pearson'), annot=True);

In [ ]:
# Hypertension percentual
df2['hypertension'].value_counts()


##### 5.2.11 - Hypotension, more risk
   False

In [ ]:
aux = df2[['hypotension', 'cardio']].groupby('hypotension').mean().reset_index()

plt.subplot(1,2,1)
sns.barplot(x='hypotension', y='cardio', data=aux);

plt.subplot(1,2,2)
sns.heatmap(df2[['hypotension', 'cardio']].corr('pearson'), annot=True);

In [ ]:
df2['hypotension'].value_counts()

##### 5.2.12 - BMI - Obesity, more risk
   True

In [ ]:
aux = df2[['bmi_level', 'cardio']].groupby('bmi_level').value_counts(normalize=True).reset_index()
print(aux)
aux = df2[['bmi_level', 'cardio']].groupby('bmi_level').mean().reset_index()
print(aux)

plt.subplot(1,2,1)
sns.barplot(x='bmi_level', y='cardio' ,data=aux);

plt.subplot(1,2,2)
sns.heatmap(df2[['bmi_level', 'cardio']].corr('pearson'), annot=True);

# aux1 = df2[(df2['cardio'] ==1)]
# aux2 = df2[(df2['cardio'] ==2) ]


##### 5.2.13 - Gender 2 has more risk
   False

In [ ]:
aux = df2[['gender', 'cardio']].groupby('gender').value_counts(normalize=True).reset_index()
print(aux)
aux = df2[['gender', 'cardio']].groupby('gender').mean().reset_index()
print(aux)

In [ ]:
plt.subplot(1,3,1)
sns.barplot(x='gender', y='cardio', data=df2)
plt.subplot(1,3,2)
sns.countplot(x='gender', hue='cardio', data=df2 );
plt.subplot(1,3,3)
sns.heatmap(df2[['gender', 'cardio']].corr('pearson'), annot=True);


##### 5.2.14 - how more weith more risk
   True

In [ ]:
aux = df2[['weight_range', 'cardio']].groupby('weight_range').value_counts(normalize=True).reset_index()
print(aux)
aux = df2[['weight_range', 'cardio']].groupby('weight_range').mean().reset_index()
print(aux)

In [ ]:
df2.isna().sum()

In [ ]:
aux = df2[['weight_range', 'cardio']].groupby('weight_range').mean().reset_index()
grid = GridSpec(2,2)

plt.subplot(grid[1,0])
sns.barplot(x='weight_range', y='cardio', data=aux);
                 
plt.subplot(grid[1,1])
plt.title('Correlation between CARDIO and WEIGHT')
pointb = pointbiserialr(df2['weight'], df2['cardio']).correlation
sns.heatmap(pd.DataFrame({'weight':pointb}, index=['cardio']), annot=True);

### 5.3 - Multivariate Analysis

In [ ]:
#pair grid
#fig, ax=plt.subplots(figsize=(6,6))
vars=['age','weight','height','hight_pressure','low_pressure','bmi']
p=pairplot(df2,vars=vars,hue='cardio',kind='reg');

In [ ]:
sns.heatmap(df2.corr(), annot=True);

## 6.0 - Data Preparation

In [ ]:
df3=df2.copy()


### 6.1 - Normalization



In [ ]:
# age
ss=StandardScaler()
df3['age'] = ss.fit_transform(df3[['age']].values)
pickle.dump(ss, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\age_scaling.pkl', 'wb'))

### 6.2 - Rescaling Robust Scaler

In [ ]:
rs = RobustScaler() #outiliers

# height         
df3['height'] = rs.fit_transform(df3[['height']].values)
pickle.dump(rs, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\height_scaling.pkl', 'wb'))

# weight
df3['weight'] = rs.fit_transform(df3[['weight']].values)
pickle.dump(rs, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\weight_scaling.pkl', 'wb'))

# difference     
df3['difference'] = rs.fit_transform(df3[['difference']].values)
pickle.dump(rs, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\difference_scaling.pkl', 'wb'))

# bmi            
df3['bmi'] = rs.fit_transform(df3[['bmi']].values)
pickle.dump(rs, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\bmi_scaling.pkl', 'wb'))


### 6.3 - Rescaling MinMaxScaler

In [ ]:
mms = MinMaxScaler()
# low_pressure   
df3['low_pressure'] = mms.fit_transform(df3[['low_pressure']].values)
pickle.dump(mms, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\low_pressure_scaling.pkl', 'wb'))

#hight_pressure
df3['hight_pressure'] = mms.fit_transform(df3[['hight_pressure']].values)
pickle.dump(mms, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\hight_pressure_scaling.pkl', 'wb'))


### 6.4 - Encoding

In [ ]:
# Nominal variables

# #df3=pd.get_dummies(df3,prefix=['gender'],columns=['gender'])

# One hot encoder
ohe = OneHotEncoder()
X=ohe.fit_transform(df3.gender.values.reshape(-1,1)).toarray()
X=pd.DataFrame(X,columns=[0,1]).rename(columns = {0:'gender_1',1:'gender_2'})
df3=pd.concat([df3,X],axis=1,join="inner")
df3=df3.drop(['gender'],axis=1)
pickle.dump(df3, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\parameters\gender_scaling.pkl', 'wb'))


## 7.0 - Feature Selection

### 7.1 Spliting after rescaling

In [ ]:
X = df3.drop(['cardio','id'], axis=1).copy()
y = df3['cardio'].copy()
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

### 7.1 - PCA

In [ ]:
X.shape

In [ ]:
pca= PCA(random_state=42)
X_pca=pca.fit_transform(StandardScaler().fit_transform(X))
pca.explained_variance_ratio_
pca.components_[0]

In [ ]:
plt.figure(figsize=(10,5) )
plt.plot( np.cumsum( pca.explained_variance_ratio_ ) )
plt.axvline( 20, color='red', linestyle='--')
plt.axhline( 0.95, color='red', linestyle='--');

In [ ]:
# change data to new space
k = np.argmax( np.cumsum( pca.explained_variance_ratio_ ) > 0.95 )
k

### 7.2 - RFE

In [ ]:
# Create linear regression object
mdl = ExtraTreesClassifier(n_estimators=100)

# Create RFE object 
rfe = RFE(mdl, n_features_to_select=k)

# Fit RFE
rfe.fit(X, y)

# Print selected features
select_features_rfe=list(X.columns[rfe.support_])
select_features_rfe

### 7.3 - Decision Tree

In [ ]:
# Create ExtraTreesClassifier object
clf = ExtraTreesClassifier(n_estimators=100)

# Fit ExtraTreesClassifier
clf.fit(X, y)

# Print feature importances
print("Feature importances: {}".format(clf.feature_importances_))

# Select top 2 features
top_indices = clf.feature_importances_.argsort()[::-1][:9]
top_features = X.columns[top_indices]
#print("Selected features: {}".format(top_features))
select_features_dt=list(top_features)
select_features_dt

### 7.4 - Boruta

In [ ]:
#!pip install boruta
from boruta import BorutaPy

x_boruta = x_train.values
y_boruta = y_train.ravel()
clf = ExtraTreesClassifier(n_estimators=100)

# Defining Boruta
boruta = BorutaPy(clf, n_estimators='auto', verbose=0, random_state=42).fit(x_boruta, y_boruta)

cols_selected = boruta.support_.tolist()
cols_selected_boruta = x_train.iloc[:, cols_selected].columns.to_list()
cols_selected_boruta

### 7.5 Features selected

In [ ]:
x_train = x_train[select_features_rfe].copy()
x_valid=x_valid[select_features_rfe].copy()

## 8.0 - Modeling

### 8.1 - Baseline

*The* baseline is the "status quo", between **55%** and **65%** of **PRECISION** Score

### 8.2 - Random Forest

In [ ]:
mdl = RandomForestClassifier(n_estimators=150, verbose=1,random_state=42).fit(x_train, y_train)
mdl_name = 'RandomForestClassifier'
y_hat = mdl.predict(x_valid)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)

df3_score_rf = metrics_score(y_valid, y_hat, mdl_name)
df3_score_rf['precision_cv'] = mean(n_scores)
df3_score_rf['precision_cv_std'] = std(n_scores)
df3_score_rf

In [ ]:
cm = confusion_matrix(y_valid, y_hat)
ConfusionMatrixDisplay(cm).plot();

In [ ]:
# from sklearn.metrics import roc_curve
# from sklearn.metrics import RocCurveDisplay

# y_score = clf.decision_function(X_test)

# fpr, tpr, _ = roc_curve(y_valid, y_hat)
# roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

### 8.3 - GBDT

In [ ]:
mdl = GradientBoostingClassifier().fit(x_train, y_train)
mdl_name ='GradientBoostingClassifier'
y_hat = mdl.predict(x_valid)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)

df3_score_gbdt = metrics_score(y_valid, y_hat, mdl_name)
df3_score_gbdt['precision_cv'] = mean(n_scores)
df3_score_gbdt['precision_cv_std'] = std(n_scores)
df3_score_gbdt

In [ ]:
cm = confusion_matrix(y_valid, y_hat)
ConfusionMatrixDisplay(cm).plot();

### 8.4 - Logistic Regression

In [ ]:
mdl = LogisticRegression(random_state=42).fit(x_train, y_train)
mdl_name = 'LogisticRegression'
y_hat = mdl.predict(x_valid)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)

df3_score_lr = metrics_score(y_valid, y_hat, mdl_name)
df3_score_lr['precision_cv'] = mean(n_scores)
df3_score_lr['precision_cv_std'] = std(n_scores)
df3_score_lr

In [ ]:
cm = confusion_matrix(y_valid, y_hat)
ConfusionMatrixDisplay(cm).plot();

### 8.5 - XGBClassifier

In [ ]:
mdl = XGBClassifier().fit(x_train, y_train)
mdl_name = 'XGBClassifier'

y_hat = mdl.predict(x_valid)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)

df3_score_xgbc = metrics_score(y_valid, y_hat, mdl_name)
df3_score_xgbc['precision_cv'] = mean(n_scores)
df3_score_xgbc['precision_cv_std'] = std(n_scores)
df3_score_xgbc

In [ ]:
cm = confusion_matrix(y_valid, y_hat)
ConfusionMatrixDisplay(cm).plot();

### 8.6 - KNClassifier

In [ ]:
mdl = KNeighborsClassifier().fit(x_train, y_train)
mdl_name = 'KNeighborsClassifier'
yhat = mdl.predict(x_valid)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)

df3_score_knc = metrics_score(y_valid, y_hat, mdl_name)
df3_score_knc['precision_cv'] = mean(n_scores)
df3_score_knc['precision_cv_std'] = std(n_scores)
df3_score_knc

In [ ]:
cm = confusion_matrix(y_valid, y_hat)
ConfusionMatrixDisplay(cm).plot();

### 8.7 - LGBMClassifier

In [ ]:
mdl = LGBMClassifier().fit(x_train, y_train)
mdl_name = 'LGBMClassifier'
y_hat = mdl.predict(x_valid)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)

df3_score_lgbmc = metrics_score(y_valid, y_hat, mdl_name)
df3_score_lgbmc['precision_cv'] = mean(n_scores)
df3_score_lgbmc['precision_cv_std'] = std(n_scores)
df3_score_lgbmc

In [ ]:
cm = confusion_matrix(y_valid, y_hat)
ConfusionMatrixDisplay(cm).plot();

## 9.0 - Metrics

### 9.1 - Validation

In [ ]:
df3_all_scores_sbs = pd.concat([df3_score_rf, df3_score_gbdt, df3_score_lr, df3_score_xgbc, df3_score_knc, df3_score_lgbmc], axis=0).reset_index(drop=True)
df3_all_scores_sbs.style.highlight_max(subset = ["accuracy",'precision','recall','F1_score','precision_cv','precision_cv_std'])

## 10.0 - Hyperparameters (HyperOpt)

**Gradient Boosting it's better**

In [ ]:
parameters = {'learning_rate':[0.01, 0.08], 
    'n_estimators':[100, 150],
    'subsample':[0.08, 0.1, 0.2, 0.6, 0.7, 0.8, 0.9, 1.0], 
    'min_samples_split':[0.3, 0.6, 0.8, 2, 3, 4],
    'min_samples_leaf':[1, 2, 3, 4], 
    'max_depth':[3, 5, 10, 15, 20, 25]}

In [ ]:
#RepeatedStratifiedKFold
MAX_EVALS = 20
df3_hyper = pd.DataFrame(columns=['Mean_precision', 'Std_precision', 'Parameters'])

for i in range(MAX_EVALS):
    print(f'evals: {i}')
    
    hp = {k:sp(v, 1)[0] for k, v in parameters.items()}
    print(f'Hyperparameters: {hp}')
    mdl = GradientBoostingClassifier(
        learning_rate=hp['learning_rate'], n_estimators=hp['n_estimators'],
        subsample=hp['subsample'], min_samples_split=hp['min_samples_split'],
        min_samples_leaf=hp['min_samples_leaf'], max_depth=hp['max_depth'],
        loss='deviance', random_state=42
        ).fit(x_train, y_train)
    print('trained...')
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    n_scores = cross_val_score(mdl, x_train, y_train, scoring='precision', cv=cv, n_jobs=-1)
    print('crossvalidation finished')
    
    df3_cv = pd.DataFrame([[mean(n_scores), std(n_scores), hp]], 
                        columns=['Mean_precision', 'Std_precision', 'Parameters'])
    
    print(df3_cv)
    df3_hyper = pd.concat([df3_hyper, df3_cv], axis=0)
    
df3_hyper

## 11.0 - Final Model

In [ ]:
learning_rate= 0.01
n_estimators= 150
subsample= 0.9
min_samples_split= 4
min_samples_leaf= 1
max_depth=5


In [ ]:
final_model = GradientBoostingClassifier(
        learning_rate=learning_rate, n_estimators=n_estimators,
        subsample=subsample, min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,max_depth=max_depth,
        loss='deviance', random_state=42
        ).fit(x_train, y_train)


pickle.dump(final_model, open(r'C:\Users\Utente77\repos\cardio_catch_disease_PA001\model\model_trained.pkl', 'wb'))

## 12.0 - Business Performance

|                                            | Avg Precision             | 
|--------------------------------------------|---------------------------|
| Gradient Boosting Classifier (Tuned HP)               | 0.756197 (+/- 0.005722) | 



In [ ]:
cost_status_quo = 1000                         ## cost of each diagnostic with operators and machines
percentual_status_quo = (0.55+0.65)/2
print(percentual_status_quo)
revenue_status_quo = int((percentual_status_quo - 0.50)/0.05) * 500
print(revenue_status_quo)

result_status_quo = revenue_status_quo - cost_status_quo

cost_future = 0
standard = 0.005722
percent_max=0.756197+0.005722
print(percent_max)
percent_min=0.756197-0.005722
print(percent_min)
percentual_future = (percent_min+percent_max)/2
revenue_future = int((percentual_future - 0.50)/0.05) * 500  
result_model_future = (revenue_future - cost_future)*70000   
final=result_model_future-result_status_quo

In [ ]:
print(result_status_quo)
print(result_model_future)
print(final)

In [ ]:
bp=pd.DataFrame(pd.concat([pd.Series(result_status_quo), pd.Series(result_model_future)])).T
bp.columns=['status_quo','result_model']
bp

Let's recap the pricing model. The price of the diagnosis, paid by the client, varies according to the precision achieved by the team of specialists.

| Exam Accuracy | Price          | Rules                                    | Example                         |
|:--------------|:---------------|:-----------------------------------------|:--------------------------------|
| Above 50%     | min \$500\.00  | \+\$500 for each additional 5% precision | Precision = 55% \-> \$1,000\.00 |
| Up to 50%     | $0\.00         | N/A                                      | N/A                             |


Our full original data set contains the records of 70,000 patients. Suppose we were to make them go through the clinic procedure to check if they have a cardiovascular disease, our model have reached a **precision** that ranges **from 0.75% to 0.76%**, which is higher than the 55% to 65% that we have on today's procedures. Thus, translating it to business numbers.

|                        | Average             | 
|:-----------------------|------------------:|
| Our model (TO BE)      | \$175,000,000  | 
| Today (AS IS)          | \$105,000,000  | 


This means that having a portfolio of 70k patients that would go through the clinical procedure to check whether they have or not a cardiovascular disease, in the **worst business scenario** the portfolio would generate a profit of **\\$171.7 million** and in the **best scenario \$186.2 million**, in contrast to today's procedure that at its best has an accuracy of 65% and would generate a total of \\$105 million, that's **a difference of \\$70.0 million (an increase of 77.3%)!**

## 13.0 - Deploy

### 13.1 - Class

In [ ]:
import pandas as pd
import numpy as np
import json
import pickle

# this file will be a handler "API"
class Cardio_Catch_Diseases(object):
    def __init__(self):
        self.home_path              = '/home/python/Documents/Projects/pa001_cardio_catch_diseases/'
        self.age_scaling            = pickle.load(open(self.home_path + 'parameters/age_scaling.pkl', 'rb'))
        self.bmi_scaling            = pickle.load(open(self.home_path + 'parameters/bmi_scaling.pkl', 'rb'))
        self.difference_scaling     = pickle.load(open(self.home_path + 'parameters/difference_scaling.pkl', 'rb'))
        self.height_scaling         = pickle.load(open(self.home_path + 'parameters/height_scaling.pkl', 'rb'))
        self.hight_pressure_scaling = pickle.load(open(self.home_path + 'parameters/hight_pressure_scaling.pkl', 'rb'))
        self.low_pressure_scaling   = pickle.load(open(self.home_path + 'parameters/low_pressure_scaling.pkl', 'rb'))
        self.weight_scaling         = pickle.load(open(self.home_path + 'parameters/weight_scaling.pkl', 'rb'))
        self.gender_encoder         = pickle.load(open(self.home_path + 'parameters/gender_scaling.pkl', 'rb'))


    
    def data_cleaning(self, df3):
        '''df3: is the input data from user on website'''


        # height - change centimeters to meters
        df3['height'] = (df3['height'] / 100)

        return df3

    def feature_engineering(self, df3):
        '''Get the dataframe from data cleaning and create a new features'''


        # feature engineering
        df3['difference'] = df3['hight_pressure'] - df3['low_pressure']
        df3['convergent'] = df3['difference'].apply(lambda x: 1 if x <= 30 else 0)
        df3['divergent'] = df3['difference'].apply(lambda x: 1 if x >= 60 else 0)
        df3['bmi'] = (df3['weight']/(df3['height'] * df3['height']))
        df3['bmi_level'] = df3['bmi'].apply(lambda x: 
                                                      'underweight' if x <= 18.5 else
                                                      'normal'      if x >  18.5 and x <= 24.9 else
                                                      'overweight'  if x >= 25.0 and x <= 29.9 else
                                                      'obesity')
        df3['hypertension'] = df3.apply(lambda x: 1 if x[4] >=140 and x[5] >= 90 else 0, axis=1)
        df3['hypotension'] = df3.apply(lambda x: 1 if x[4] <=90 and x[5] <= 60 else 0, axis=1)
        df3['age_range'] = df3['age'].apply(lambda x: 'age_0_50'    if x <= 50 
                                                           else 'age_51_59'   if x >= 51 and x <= 59 
                                                           else 'age_60_over' if x >= 60 
                                                           else x)

        df3['weight_range'] = df3['weight'].apply(
            lambda x:   0.50 if x <= 50  else
                        0.75 if x >= 51  and x <= 75 else
                        1.00 if x >= 76  and x <= 100 else
                        1.25 if x >= 101 and x <= 125 else
                        1.50 if x >= 126 and x <= 150 else
                        1.75 if x >= 151 and x <= 175 else 2.00)

        # Data Filtering rows, values or columns
        df3 = df3.drop('id', axis=1)

        return df3


    def data_rescaling(self, df3):
        '''
        Rescaling data
        Return: json file with x_test data to input the prediction
        '''


        # Transforming data to will be in correct data scaler to input at model
        df3['age']            = self.age_scaling.transform(df3[['age']].values)
        df3['height']         = self.height_scaling.transform(df3[['height']].values)
        df3['weight']         = self.weight_scaling.transform(df3[['weight']].values)
        df3['low_pressure']   = self.low_pressure_scaling.transform(df3[['low_pressure']].values)
        df3['hight_pressure'] = self.hight_pressure_scaling.transform(df3[['hight_pressure']].values)
        df3['difference']     = self.difference_scaling.transform(df3[['difference']].values)
        df3['bmi']            = self.bmi_scaling.transform(df3[['bmi']].values)
        df3['gender']         = self.gender_scaling.transform(df3[['gender']].values)
        df3['age_range']      = df3['age_range'].map(
            {'age_0_50':1, 'age_51_59':1, 'age_60_over':3})
        df3['bmi_level']      = df3['bmi_level'].map(
            {'underweight':1, 'normal':2, 'overweight':3, 'obesity':4})

        # Deleting target
        x_test = df3.drop('cardio', axis=1).copy()
        #data_json = json.dumps(x_test.to_dict(orient='records')) # converto to json

        return x_test

### 13.2 - Input Simulation

In [ ]:
import pandas as pd

def input_simulation():
    # inputs from user
    
    age = input('age: ')
    gender = int(input('gender: '))
    height = int(input('height: '))
    weight = float(input('weight: '))
    ap_hi = int(input('hight_pressure: '))
    ap_lo = int(input('low_pressure: '))
    cholesterol = int(input('cholesterol (1,2 or 3): '))
    gluc = int(input('glucose (1,2 or 3): '))
    smoke = int(input('smoker: '))
    alco = int(input('alcohol: '))
    active = input('active: ')

    # replacing columns names
    new_col = ['id', 'age', 'gender', 'height', 'weight', 'hight_pressure', 'low_pressure',
               'cholesterol', 'glucose', 'smoker', 'alcohol', 'active', 'cardio']

    # getting data and converting to dataframe
    df3_inputed = [[1, age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke, alco, active, 0,]]
    df3 = pd.DataFrame(df3_inputed, columns=new_col)

    # datatypes transformations
    df3['age'] = df3['age'].astype('int64')
    df3['gender'] = df3['gender'].astype('int64')
    df3['height'] = df3['height'].astype('int64')
    df3['weight'] = df3['weight'].astype('float64')
    df3['hight_pressure'] = df3['hight_pressure'].astype('int64') 
    df3['low_pressure'] = df3['low_pressure'].astype('int64') 
    df3['cholesterol'] = df3['cholesterol'].astype('int64')
    df3['glucose'] = df3['glucose'].astype('int64')
    df3['smoker'] = df3['smoker'].astype('int64') 
    df3['alcohol'] = df3['alcohol'].astype('int64')
    df3['active'] = df3['active'].astype('int64')
    df3['cardio'] = df3['cardio'].astype('int64')

    return json.dumps(df3.to_dict(orient='records'))

### 13.3 - API / Handler

In [ ]:
import pickle
import pandas as pd
import numpy as np
from flask import Flask, request, Response
from ccd.ccd import Cardio_Catch_Diseases
import json

# loading model trained from pickle file
with open('/home/python/Documents/Projects/pa001_cardio_catch_diseases/model/model_trained.pkl', 'rb') as file:
    model = pickle.load(file)

# initialized API
app = Flask(__name__)

@app.route('/ccd/predict', methods=['POST'])
def ccd_predict():
    '''
    ccd: Cardio Catch Diseases
    This function it will be create the API request, where the user will be input the user data that 
    will used to data_cleaning(), data_engineering(), data_scaler() 
    this functions will be return the dataframe 'x_est', that will be used to predict
    Return: Prediction "array"
    '''
    
    test_json = request.get_json()
        
    if test_json:
        if isinstance(test_json, dict): # Unique Example
            test_raw = pd.DataFrame(test_json, index=[0])
        else: # multiple Example
            test_raw = pd.DataFrame(test_json, columns=test_json[0].keys())

        # Instantiate CCD class
        pipeline = Cardio_Catch_Diseases()
        # cleaning data
        test_raw_cleaned = pipeline.data_cleaning(test_raw)
        # creating derivates to new features
        test_raw_features = pipeline.feature_engineering(test_raw_cleaned)        
        
        # preparing data to model
        test_raw_selected = pipeline.data_rescaling(test_raw_features)
        
        y_hat = model.predict(test_raw_selected)
        test_raw['prediction'] = y_hat.copy()
        return test_raw.to_json(orient='records')
    
    else:
        return Response( "{}", status=200, mimetype='application/json')

    
if __name__=='__main__':
    app.run('0.0.0.0')

### 13.4 - API Request Tester

In [ ]:
import requests

# Input data user
data = input_simulation()

# API request
url = "https://ccd-model.herokuapp.com/ccd/predict"
header = {'Content-type': 'application/json'}

response_ccd = requests.post(url, data=data, headers=header)
print(f'Status Code: {response_ccd.status_code}')

df3_response = pd.DataFrame(response_ccd.json(), columns=response_ccd.json()[0].keys())

# Loading json to Dataframe
df3_new = pd.DataFrame(json.loads(data))

# converting to back the original datatype
df3_new['age'] = df3_new['age'] * 365

# getting prediction
df3_new['cardio'] = df3_response['cardio'].copy()

# getting current time in seconds since the Epoch.
# convert to normal date use time.ctime(time.time())
df3_new['id'] = time.time()

# Insert into database
insert_database_values(df3_new)

df3_response

## 14.0 - Dashboard - PowerBI

In [ ]:
# Connecting sqlite database
import pandas as pd
from sqlalchemy import create_engine

new_col = ['id', 'age', 'gender', 'height', 'weight', 'hight_pressure', 'low_pressure',
           'cholesterol', 'glucose', 'smoker', 'alcohol', 'active', 'cardio']

engine = create_engine('sqlite:///Y:\\pa001_cardio_catch_diseases\\ccd.db', echo=False)
df3 = engine.execute('SELECT * FROM patients').fetchall()
df3 = pd.DataFrame(df3, columns=new_col)

# Data Preparation/transformation

